In [1]:
import pandas as pd

In [25]:

df = pd.read_json("/shared/3/projects/benlitterer/podcastData/annotation/annotated_output/annotated_instances.json", orient="records", lines=True)

In [26]:
df = df.explode(["span_annotations"]).reset_index(drop=True)
dictList = df["span_annotations"]

In [27]:
#get this to long format 
spanAnnots = pd.DataFrame.from_records(dictList).reset_index(drop=True)

In [29]:
df = pd.concat([df, spanAnnots], axis=1)

In [31]:
df["annotation"].value_counts()

annotation
host     27
guest    26
Name: count, dtype: int64

In [41]:
print(f'Number of episodes with host: {len(df.loc[df["annotation"] == "host", "id"].unique())}') 
print(f'Number of episodes with guest: {len(df.loc[df["annotation"] == "guest", "id"].unique())}') 
print(f'Number of episodes with either: {len(df["id"].unique())}') 

Number of episodes with host: 19
Number of episodes with guest: 13
Number of episodes with guest: 24


In [54]:
jHost = df.loc[df["annotation"] == "host", "id"].unique()
jGuest = df.loc[df["annotation"] == "guest", "id"].unique()
print(f"Number of episodes with both: {len(set(jHost).intersection(set(jGuest)))}")


Number of episodes with both: 8


,id,displayed_text,label_annotations,span_annotations,behavioral_data,start,end,span,annotation
37,https://r.zencastr.com/r/www.buzzsprout.com/97...,<strong>Description:</strong><br>THIS EPISODE....,{},"{'start': 71, 'end': 86, 'span': 'Sidney Cachu...",{'time_string': 'Time spent: 0d 0h 0m 44s '},71,86,Sidney Cachuela,guest
